In [15]:
# Initialize the random number generator
rng = MersenneTwister(2016);

In [16]:
include("gees.jl")

gees_testsuite (generic function with 1 method)

In [17]:
n = 4
Λ = diagm(Float32[2.0^(-i) for i=0:n-3])
A = zeros(n,n)
A[1:n-2,1:n-2] = Λ
A[n-1:n,n-1:n] = [0 1; -1 0]
X = rand(rng, n, n)

# Λ = diagm(Float32[2.0^(-i) for i=0:n-1])
# A = Λ
# X, = qr(rand(rng, n, n))

A = X * A / X

println("Matrix A")
pretty_print(A)
Λ = eigenvalue_sorted(A)

Matrix A
 -2.3e+00  1.2e+00  2.8e+00 -4.7e+00
 -3.3e+00  1.1e+00  3.9e+00 -5.9e+00
 -1.3e+00 -1.2e+00  3.9e+00 -4.5e+00
 -6.1e-01 -5.9e-01  1.4e+00 -1.3e+00


4-element Array{Complex{Float64},1}:
         0.5+0.0im
 4.30211e-16-1.0im
 4.30211e-16+1.0im
         1.0+0.0im

In [18]:
# Applying Householder reflections to make matrix A upper Hessenberg
gehrd!(A)
A0 = copy(A)

4x4 Array{Float64,2}:
 -2.2636   -1.27441  0.173003   -5.44537 
  3.54181   1.10846  0.921535    8.81452 
  0.0      -1.03282  1.67318     2.69294 
  0.0       0.0      0.0120129   0.981953

In [19]:
# Checking that A is upper Hessenberg now
println("Norm for lower part of matrix = ",norm(tril(A,-2)))
println("Matrix A")
A

Norm for lower part of matrix = 0.0
Matrix A


4x4 Array{Float64,2}:
 -2.2636   -1.27441  0.173003   -5.44537 
  3.54181   1.10846  0.921535    8.81452 
  0.0      -1.03282  1.67318     2.69294 
  0.0       0.0      0.0120129   0.981953

In [20]:
# Checking that the eigenvalues are the same
D = eigenvalue_sorted(A)
println(norm(D-Λ))

5.718490613591023e-15


In [21]:
# Sequential version of QR iteration using Givens rotations
givens_QR_iteration_s!(A)
A

4x4 Array{Float64,2}:
 -0.89831   2.47099   -2.65777    -10.6821  
 -0.958563  1.36666    1.5524      -2.36684 
  0.0       0.108168   0.158493    -1.06853 
  0.0       0.0       -0.0884887    0.873154

In [22]:
# Testing the result
D = eigenvalue_sorted(A)
@show norm(tril(A,-2))
@show norm(D-Λ)

norm(tril(A,-2)) = 0.0
norm(D - Λ) = 5.69824738883827e-15


5.69824738883827e-15

In [23]:
# QR iteration with bulge chasing
A = copy(A0)
givens_QR_iteration!(A)
A

4x4 Array{Float64,2}:
 -0.89831      2.47099      -2.65777    -10.6821  
 -0.958563     1.36666       1.5524      -2.36684 
  3.33067e-16  0.108168      0.158493    -1.06853 
  0.0          8.67362e-18  -0.0884887    0.873154

In [24]:
# Testing the result
D = eigenvalue_sorted(A)
@show norm(tril(A,-2))
@show norm(D-Λ)

norm(tril(A,-2)) = 3.3306690738754696e-16
norm(D - Λ) = 5.123140958641412e-15


5.123140958641412e-15

In [25]:
# Single step of QR iteration with double real shift
A = copy(A0)
gees_single_step!(A, false) # false: exceptional_shift is not applied
A

4x4 Array{Float64,2}:
 -0.486195     -3.37541    3.46473     10.471   
  0.341144      0.370591   2.38897      0.383429
 -5.55112e-17  -0.041794   0.615662    -0.257046
  4.33681e-19   0.0       -8.13185e-5   0.999942

In [26]:
# Testing the result
D = eigenvalue_sorted(A)
@show norm(tril(A,-2))
@show norm(D-Λ)

norm(tril(A,-2)) = 5.551284527130373e-17
norm(D - Λ) = 4.67115133410009e-15


4.67115133410009e-15

In [27]:
err = gees_testsuite()


 *** Test no 1 ***
Size of matrix 1
norm(D0 - D1) = 0.0
norm(D1 - D2) = 0.0

 *** Test no 2 ***
Size of matrix 2
norm(D0 - D1) = 0.0
norm(D1 - D2) = 0.0

 *** Test no 3 ***
Size of matrix 3
norm(D0 - D1) = 1.336327202703161e-15
norm(D1 - D2) = 2.237726045655905e-16

 *** Test no 4 ***
Size of matrix 4
norm(D0 - D1) = 3.647298083769891e-15
norm(D1 - D2) = 4.458205583648681e-16

 *** Test no 5 ***
Size of matrix 3
Exceptional shift
norm(D0 - D1) = 3.6821932062951477e-16
norm(D1 - D2) = 0.0

 *** Test no 6 ***
Size of matrix 4
Exceptional shift
norm(D0 - D1) = 8.0051527712734e-16
norm(D1 - D2) = 3.920818324662287e-17

 *** Test no 7 ***
Size of matrix 7
norm(D0 - D1) = 2.3376920389573177e-14
norm(D1 - D2) = 3.863026258982364e-15

 *** Test no 8 ***
Size of matrix 8
norm(D0 - D1) = 2.3675684962721063e-14
norm(D1 - D2) = 5.659279512727921e-15

 *** Test no 9 ***
Size of matrix 9
Exceptional shift
norm(D0 - D1) = 6.655153300303468e-14
norm(D1 - D2) = 4.610909566101516e-14

 *** Test no 10 *

50-element Array{Float64,1}:
 0.0        
 0.0        
 1.11336e-15
 4.07941e-15
 3.68219e-16
 7.93734e-16
 2.65992e-14
 2.7647e-14 
 1.10636e-13
 2.65459e-14
 0.0        
 0.0        
 3.68219e-16
 ⋮          
 2.52591e-15
 3.54285e-14
 0.0        
 1.11022e-16
 7.02167e-16
 1.5862e-15 
 1.6486e-15 
 2.53656e-15
 1.00554e-15
 1.62601e-15
 2.26903e-15
 4.11251e-15

In [28]:
# Plot error at each step of the test suite
using Plots
plotlyjs()
plot(err,yscale=:log10,left_margin=40mm,lab="Error")
xlabel!("Test index")
ylabel!("Error")